<a href="https://colab.research.google.com/github/Dannykik/ULPGL_work/blob/main/L3_work/detection_anomalie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Chemin du fichier (Google Drive)
file_path = "/content/drive/MyDrive/Mon_memoire_L3/UCSD_Anomaly_Dataset.tar.gz"

# Extraire l'archive dans /content
!tar -xvzf "$file_path" -C /content

In [ ]:
import os
os.listdir('/content')

In [ ]:
os.listdir('/content/UCSD_Anomaly_Dataset.v1p2')

In [ ]:

train_path = '/content/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train'
print(os.listdir(train_path)[:5])

In [ ]:
test_path = '/content/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test'
print(os.listdir(test_path)[:5])

#Activer le GPU (vérification)

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

#Charger les images (proprement)

In [ ]:
import os
import cv2
import numpy as np

IMG_SIZE = 128

def load_images_from_folder(folder):
    images = []
    for subfolder in sorted(os.listdir(folder)):
        sub_path = os.path.join(folder, subfolder)
        if os.path.isdir(sub_path):
            for file in sorted(os.listdir(sub_path)):
                if file.endswith('.tif'):
                    img_path = os.path.join(sub_path, file)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                    img = img / 255.0
                    images.append(img)
    return np.array(images)

train_folder = '/content/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train'
train_images = load_images_from_folder(train_folder)

print("Nombre d'images d'entraînement :", train_images.shape)

#Construire l’Autoencoder CNN

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

train_images = train_images.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

input_img = Input(shape=(IMG_SIZE, IMG_SIZE, 1))

# Encoder
x = Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same')(x)

# Decoder
x = Conv2D(64, (3,3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
decoded = Conv2D(1, (3,3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.summary()


#Entraînement

In [ ]:
autoencoder.fit(
    train_images,
    train_images,
    epochs=10,
    batch_size=16,
    shuffle=True
)

#Détection des anomalies

In [ ]:
test_folder = '/content/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test'
test_images = load_images_from_folder(test_folder)
test_images = test_images.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

print("Nombre d'images de test :", test_images.shape)


#Reconstruction avec l’autoencoder

In [ ]:
reconstructed = autoencoder.predict(test_images)

#Calculer l’erreur de reconstruction

In [ ]:
mse = np.mean(np.power(test_images - reconstructed, 2), axis=(1,2,3))

#Définir un seuil

In [ ]:
threshold = np.mean(mse) + 3*np.std(mse)
print("Seuil d'anomalie :", threshold)

#Identifier les anomalies

In [ ]:
anomalies = mse > threshold
print("Nombre d'anomalies détectées :", np.sum(anomalies))

#Visualiser quelques résultats

In [ ]:
import matplotlib.pyplot as plt

for i in range(5):
    plt.figure(figsize=(6,3))
    # Image originale
    plt.subplot(1,2,1)
    plt.imshow(test_images[i].reshape(IMG_SIZE,IMG_SIZE), cmap='gray')
    plt.title("Originale")
    plt.axis('off')

    # Image reconstruite
    plt.subplot(1,2,2)
    plt.imshow(reconstructed[i].reshape(IMG_SIZE,IMG_SIZE), cmap='gray')
    plt.title(f"Reconstruit\nErreur={mse[i]:.4f}")
    plt.axis('off')

    plt.show()

In [ ]:
# Sauvegarder
autoencoder.save('/content/autoencoder_ucsd.h5')

In [ ]:
drive.mount('/content/drive')

autoencoder.save('/content/drive/MyDrive/Mon_memoire_L3/autoencoder_ucsd.keras')

In [ ]:
autoencoder.save('/content/drive/MyDrive/Mon_memoire_L3/autoencoder_ucsd.h5')

# Model pour la detection d'objets

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.6 MB/s eta 0:00:00


In [2]:
import torch
print("GPU disponible :", torch.cuda.is_available())


GPU disponible : True


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from ultralytics import YOLO

# Modèle pré-entraîné sur COCO
model = YOLO("yolov8n.pt")   # version légère (rapide)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
import os

dataset_path = "/content/drive/MyDrive/Mon_memoire_L3/New weapon dataset"
os.listdir(dataset_path)

['test', 'train', 'README.dataset.txt', 'README.roboflow.txt', 'data.yaml']

In [12]:
!cp "/content/drive/MyDrive/Mon_memoire_L3/New weapon detection/weapon detection.v1i.yolov8.zip" /content/

In [14]:
!unzip -q "/content/weapon detection.v1i.yolov8.zip" -d /content/dataset

replace /content/dataset/README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/dataset/README.roboflow.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/dataset/data.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/dataset/test/images/-12_14_2022-9_13_49-PM_png.rf.22cdc174b30e6b5fa21cd6d9c3e0c331.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [15]:
!ls /content/dataset


data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid


In [ ]:
from ultralytics import YOLO

# Charger modèle pré-entraîné (Transfer Learning)
model = YOLO("yolov8n.pt")

model.train(
    data="/content/dataset/data.yaml",  # Dataset local
    epochs=50,
    imgsz=640,
    batch=16,
    optimizer="AdamW",      # Optimiseur moderne
    lr0=0.001,              # Learning rate initial
    weight_decay=0.0005,    # L2 Regularization
    patience=20,            # Early stopping
    augment=True,           # Data augmentation
    device=0                # GPU
)


Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=20, perspective=0.0, 